<h1>Using TensorFlow With GPU</h1>
<p>Assuming you have an NVIDIA GPU with Cuda Compute Capability 3.0 or above...</p>
<p>Build TensorFlow from <a href="https://www.tensorflow.org/versions/master/get_started/os_setup.html#source">source</a> and configure it using the following command:</p>

In [1]:
%%bash
# cd to tensorflow root, do the following... The unofficial setting lets you use 3.0 GPUs instead of minimum 3.5
# TF_UNOFFICIAL_SETTING=1 ./configure

<p>Note that the above has some interactive prompts you need to fill out, so you can't do it from within this notebook. Then create a pip install like this:</p>

In [2]:
%%bash
# Make sure you're using python 2.7
python --version
# download and install tensorflow with gpu capability (from the pip package you build from source!!!)
# see: https://www.tensorflow.org/versions/master/get_started/os_setup.html#create-pip

# ====================== UNCOMMENT THIS LINE ===================== #
# pip install /tmp/tensorflow_pkg/tensorflow-0.5.0-py2-none-any.whl
#
# Note: the name of the .whl may change in the future...
#
# Or if you're doing it from another machine...
# ====================== UNCOMMENT THIS LINE ===================== #
# pip install tensorflow
#
# Or, if that doesn't work
# ====================== UNCOMMENT THIS LINE ===================== #
# conda install -c https://conda.anaconda.org/jjhelmus tensorflow

Python 2.7.11 :: Anaconda 2.4.0 (x86_64)


<p>Now your code will run through the damn GPU from your iPython notebook. Sick, huh? Now do this:</p>

In [3]:
import numpy as np
import tensorflow as tf
CPU = "/cpu:0"
#GPU = "/gpu:0"
GPU = "/cpu:0"

<h2>Computational Graphs with TensorFlow</h2>
<p>TensorFlow uses graphs to define computations. You create constants and operations, and using a <code>Session()</code> object, allow TesnorFlow to automatically handle the overhead of allocating resources and calling external libraries for you. When the session finishes, the resources are freed and the session terminates.</p>

In [4]:
# some constants and an operation (variables...)
matrix1 = tf.constant([[3.,3.]]) # 1 x 2 matrix
matrix2 = tf.constant([[2.],[2.]]) # 2 x 1 matrix
product = tf.matmul(matrix1, matrix2) # (1 x 2) * (2 x 1)

# create/run the session
sess = tf.Session()
result = sess.run(product)

print(result)

#close session
sess.close()

[[ 12.]]


<p>Trying the above using a <code>with</code> block...</p>

In [5]:
with tf.Session() as sess:
    result = sess.run(product)
    print(result)

[[ 12.]]


<p>You can also run it on a GPU or CPU by design:</p>

In [6]:
def run_on_dev(dev="/gpu:0"):
    with tf.Session() as sess:
        with tf.device(dev):
            A = tf.constant([[3.,3.]])
            B = tf.constant([[2.],[2.]])
            product = tf.matmul(A, B)
            result = sess.run(product)
            print(result)

dev1 = CPU
dev2 = GPU
run_on_dev(dev1)
run_on_dev(dev2)

[[ 12.]]
[[ 12.]]


## Interactive Session

In [7]:
# create interactive session
sess = tf.InteractiveSession()
x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# initialize x
x.initializer.run()

# add an op to subtract a from x
sub = tf.sub(x, a)
print(sub.eval())

sess.close()

[-2. -1.]


### Data Types

In [8]:
# floats
# print(tf.float32)
# print(tf.float64)

# ints
# print(tf.int64)
# print(tf.int32)
# print(tf.int16)
# print(tf.int8)
# print(tf.uint8)

# other
# print(tf.string)
# print(tf.bool)
# print(tf.complex64)

# quantized
# print(tf.qint32)
# print(tf.qint8)
# print(tf.quint8)

### Device Allocation & Logging

In [9]:
def dev_log(dev):
    with tf.device(dev):
        a = tf.constant([1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0], shape=[9,1], name='a')
        b = tf.constant([1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0], shape=[1,9], name='b')
        c = tf.matmul(a, b)
    # run
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess: 
        print(sess.run(c))

In [10]:
# run on CPU
dev_log(CPU)

# run on GPU
dev_log(GPU)

[[  1.   2.   3.   4.   5.   6.   7.   8.   9.]
 [  2.   4.   6.   8.  10.  12.  14.  16.  18.]
 [  3.   6.   9.  12.  15.  18.  21.  24.  27.]
 [  4.   8.  12.  16.  20.  24.  28.  32.  36.]
 [  5.  10.  15.  20.  25.  30.  35.  40.  45.]
 [  6.  12.  18.  24.  30.  36.  42.  48.  54.]
 [  7.  14.  21.  28.  35.  42.  49.  56.  63.]
 [  8.  16.  24.  32.  40.  48.  56.  64.  72.]
 [  9.  18.  27.  36.  45.  54.  63.  72.  81.]]
[[  1.   2.   3.   4.   5.   6.   7.   8.   9.]
 [  2.   4.   6.   8.  10.  12.  14.  16.  18.]
 [  3.   6.   9.  12.  15.  18.  21.  24.  27.]
 [  4.   8.  12.  16.  20.  24.  28.  32.  36.]
 [  5.  10.  15.  20.  25.  30.  35.  40.  45.]
 [  6.  12.  18.  24.  30.  36.  42.  48.  54.]
 [  7.  14.  21.  28.  35.  42.  49.  56.  63.]
 [  8.  16.  24.  32.  40.  48.  56.  64.  72.]
 [  9.  18.  27.  36.  45.  54.  63.  72.  81.]]


In [11]:
def big_tensor_multiply(dev,dim=1000):
    with tf.device(dev):
        a = tf.constant(np.random.rand(dim,dim), shape=[dim,dim], name='a')
        b = tf.constant(np.random.rand(dim,dim), shape=[dim,dim], name='b')
        c = tf.matmul(a, b)
        d = tf.matrix_inverse(c)
    # soft_placement allows tensorflow to allocate ops to device of its choice
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        print(sess.run(d))

In [12]:
big_tensor_multiply(GPU)

[[-31.38444859 -19.09501874 -46.73663672 ..., -14.53397292 -20.65020629
    6.93230569]
 [  7.84727277   3.89965026  10.73314382 ...,   3.65827177   5.22387857
   -1.67060043]
 [ -9.57701398  -6.02240057 -14.43602501 ...,  -4.46672842  -6.37462149
    1.9046587 ]
 ..., 
 [-28.75823057 -17.74147636 -42.75766215 ..., -13.07948493 -19.36042455
    6.28890704]
 [ 10.00818264   6.44302041  15.06993653 ...,   4.56704431   6.73123929
   -2.06101705]
 [ -0.77141348  -1.3193186   -2.10344004 ...,  -0.3360398   -0.23916448
   -0.07641594]]


In [13]:
def use_multiple_devices(devices):
    for d in devices:
        print("Using device: " + d)
        big_tensor_multiply(d)            

In [14]:
use_multiple_devices([GPU,CPU])

Using device: /cpu:0
[[ 0.07679646 -0.27353408 -0.12751289 ...,  0.0856424  -1.13138416
  -0.03328873]
 [ 0.36858419 -0.36931438  1.14990034 ...,  0.37799874  0.68649865
  -0.27673902]
 [ 0.61033404  0.20069893  0.57896095 ..., -0.14516779  0.60370352
  -0.29194378]
 ..., 
 [-0.24836041  0.05711571  0.41464966 ...,  0.08327197  0.64584404
   0.41700414]
 [-0.52815259 -0.64138573 -1.29735189 ...,  0.34245771 -2.52806726
  -0.10744348]
 [-0.74631385 -0.46004439 -0.67331659 ...,  0.25574346 -1.25555572
  -0.01976877]]
Using device: /cpu:0
[[ 0.01552621 -0.46725361 -0.04231887 ..., -0.06590554  0.16044502
  -0.14831765]
 [-0.21978828  0.52453695 -1.41511505 ...,  0.45818407  0.57338101
  -0.80137229]
 [ 0.18339134 -0.10927796  1.02021297 ..., -0.28026921 -0.5857836
   0.60276394]
 ..., 
 [-0.11368047  0.39952238 -0.34046508 ...,  0.20978883  0.05225077
  -0.11854756]
 [ 0.12902199 -0.58843207  0.97268844 ..., -0.46493589 -0.40401439
   0.54104254]
 [ 0.21079995 -0.1828678   0.56807836 ...,

## Using Variables

In [15]:
def counter_step(step):
    # really overblown counter
    state = tf.Variable(0, name="counter")
    
    # val 1
    one = tf.constant(step)
    
    # val step + state
    new_val = tf.add(state, one)
    
    # update assign operation
    update = tf.assign(state, new_val)

    # graph launch
    init_operation = tf.initialize_all_variables()

    # run graph
    with tf.Session() as sess:
        sess.run(init_operation)
        print(sess.run(state))
        for _ in range(3):
            sess.run(update)
            print(sess.run(state))

In [16]:
counter_step(4)

0
4
8
12


## Simple Regression Example

In [17]:
x_data = np.random.rand(100).astype("float32")
y_data = x_data * 0.1 + 0.3

# Weights
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

# Bias
b = tf.Variable(tf.zeros([1]))

# Layer function
y = W * x_data + b

# MSE
loss = tf.reduce_mean(tf.square(y - y_data))

# SGD
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# initialize variables...
init = tf.initialize_all_variables()

with tf.Session() as sess:
    with tf.device(GPU):
        sess.run(init)
        for step in xrange(201):
            sess.run(train)
            if step % 20 == 0:
                print(step, sess.run(W), sess.run(b))

(0, array([ 0.16752651], dtype=float32), array([ 0.36398095], dtype=float32))
(20, array([ 0.10501249], dtype=float32), array([ 0.29727489], dtype=float32))
(40, array([ 0.10099418], dtype=float32), array([ 0.29945952], dtype=float32))
(60, array([ 0.10019718], dtype=float32), array([ 0.29989281], dtype=float32))
(80, array([ 0.1000391], dtype=float32), array([ 0.29997876], dtype=float32))
(100, array([ 0.10000775], dtype=float32), array([ 0.29999581], dtype=float32))
(120, array([ 0.10000155], dtype=float32), array([ 0.29999918], dtype=float32))
(140, array([ 0.10000031], dtype=float32), array([ 0.29999983], dtype=float32))
(160, array([ 0.10000009], dtype=float32), array([ 0.29999995], dtype=float32))
(180, array([ 0.10000009], dtype=float32), array([ 0.29999995], dtype=float32))
(200, array([ 0.10000009], dtype=float32), array([ 0.29999995], dtype=float32))


<h2>MNIST example: Handwritten Digit Recognition</h2>
<p>First, to download and install data for MNIST dataset. This code will be reused later as well...</p>

In [18]:
import input_data # comes from the file provided in the tutorial...
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# mnist.train -- training data
# mnist.test -- testing data
# mnist.train.images -- training images
# mnist.train.labels -- training labesls

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
print("Here's what the training data looks like:\n")
print(mnist.train.images)
print("\nNum images: " + str(len(mnist.train.images)))
print("Num labels: " + str(len(mnist.train.labels)))

Here's what the training data looks like:

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]

Num images: 55000
Num labels: 55000


<h2>Model Parameters</h2>
<p>In this example we will only use a single layer model...</p>

In [20]:
# create an input vector for flattened images...
x = tf.placeholder(tf.float32, [None, 784])

# weight matrix 784 x 10 
W = tf.Variable(tf.zeros([784,10]))

# Biases
b = tf.Variable(tf.zeros([10]))

### Softmax Regression

In [21]:
# y = output = softmax(Sum(W * x) + b)
y = tf.nn.softmax(tf.matmul(x, W) + b)

### Cross-Entropy Output
General Form of the Cross Entropy loss function
\begin{align}
H_{y^{\prime}}\left(y\right) &= -\sum_i  \ y^{\prime}_i \ log\left(y_i\right)
\end{align}

In [22]:
# OP: Truth value
t = tf.placeholder(tf.float32, [None, 10])

# OP: Loss function
cross_entropy = -tf.reduce_sum(t * tf.log(y))

### Backpropagation Training

In [23]:
# OP: GD optimization
learning_rate = 0.01
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

### Initialization

In [24]:
# OP: initialize stuff. duh.
init = tf.initialize_all_variables()

### Training & Testing

In [25]:
# 1000 training iterations
def train_test(dev):
    batch_size = 100
    num_epochs = 1000

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
        with tf.device(dev):
            # Run the session
            sess.run(init)
            for i in range(num_epochs):
                # periodic print out
                if i % (num_epochs/10.0) == 0: print("Epoch: " + str(i) + "...")
                batch_inputs, truth_values = mnist.train.next_batch(batch_size)
                sess.run(train_step, feed_dict={x: batch_inputs, t: truth_values})

            # OP: compare truth values to predictions
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))

            # OP: calculate accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

            # RUN: print the accuracy
            test_result = sess.run(accuracy, feed_dict={x: mnist.test.images, t: mnist.test.labels})
            print("Accuracy on Test set: " + str(test_result))

In [26]:
train_test(GPU)

Epoch: 0...
Epoch: 100...
Epoch: 200...
Epoch: 300...
Epoch: 400...
Epoch: 500...
Epoch: 600...
Epoch: 700...
Epoch: 800...
Epoch: 900...
Accuracy on Test set: 0.9151


# Using a Deeper Model: MNIST
But first a recap...

In [27]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [28]:
# Input & Truth Vector
def get_placeholders():
    x = tf.placeholder("float",shape=[None,784])
    t = tf.placeholder("float",shape=[None,10])
    return (x, t)

# Weights & Bias
def get_model_params():
    W = tf.Variable(tf.zeros([784,10]))
    b = tf.Variable(tf.zeros([10]))
    return (W, b)

# Softmax Layer
def get_softmax_layer(x, W, b):
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cost Function
def get_cross_entropy_function(t, y):
    return -tf.reduce_sum(t * tf.log(y))

# Training module
def get_training_module(learning_rate, cost_function):
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

# Test the model
def do_test_model(inputs, outputs, truth):
    correct_prediction = tf.equal(tf.argmax(outputs,1), tf.argmax(truth,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    test_accuracy = accuracy.eval(feed_dict={inputs: mnist.test.images, truth: mnist.test.labels})
    print("Test Accuracy: " + str(test_accuracy))

# Training iterations
def do_train_model(training_algo, input_values, truth_values, batch_size, num_epochs):
    for i in range(num_epochs):
        if i % (num_epochs/10) == 0: print("Epoch " + str(i) + "...")
        batch = mnist.train.next_batch(batch_size)
        training_algo.run(feed_dict={input_values: batch[0], truth_values: batch[1]})

# Train/test
def do_train_test(learning_rate, batch_size, num_epochs):
    x, t = get_placeholders()
    W, b = get_model_params()
    y = get_softmax_layer(x, W, b)
    ce = get_cross_entropy_function(t, y)
    training_algo = get_training_module(learning_rate, ce)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        do_train_model(training_algo, x, t, batch_size, num_epochs)
        do_test_model(x, y, t)

In [29]:
learning_rate = 0.01
batch_size = 100
num_epochs = 1000
do_train_test(learning_rate, batch_size, num_epochs)

Epoch 0...
Epoch 100...
Epoch 200...
Epoch 300...
Epoch 400...
Epoch 500...
Epoch 600...
Epoch 700...
Epoch 800...
Epoch 900...
Test Accuracy: 0.9118


## Convolutional Model
Parameters and placeholders...

In [30]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Inputs and truth placeholder
x, t = get_placeholders()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Functions we'll need to do convolutions, etc...

In [31]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional Layer
This will create a convolution layer with 32 filters, each being a 5x5 pixel patch. The shape will therefore be [5,5,1,32] which indicates the size of our filters, the number of input channels (1), and the number of output channels (32). There is also a bias vector for each output channel, so a 32-dim vector of bias terms. 

In [32]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply this layer, we reshape it to a 4d tensor, with the 2nd and 3rd dimensions corresponding to the image width and heigh, and the final dimension to the number of color channels. (1 for greyscale). 

In [33]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

Convolve x_image with the weight tensor, add bias, and apply ReLU, and finally a max pool...

In [34]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Layer
64 features for each 5x5 patch.

In [35]:
# Weight & Bias
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

# ReLU activation function & Max-pooling layer
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densly Connected Layer
Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [36]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# pooling layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer
Finally, we add a softmax layer, just like for the one layer softmax regression above.

In [37]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = get_softmax_layer(h_fc1_drop, W_fc2, b_fc2)

## Train and Test

In [38]:
# Cross Entropy loss function
ce = get_cross_entropy_function(t, y_conv)

# Function to optimize
training_algo = tf.train.AdamOptimizer(1e-4).minimize(ce)

# Correct Prediction
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(t, 1))

# Accuracy calculation
accuracy_calc = tf.reduce_mean(tf.cast(correct_prediction, "float"))

Training method...

In [ ]:
# Run the Training...
def run_convolutional_mnist_model(training_algo, accuracy_calc, batch_size):
    config=tf.ConfigProto(allow_soft_placement=True)
    #config.gpu_options.allocator_type = 'BFC'
    with tf.Session(config=config) as sess:
        sess.run(tf.initialize_all_variables())
        for i in range(2000):
            batch = mnist.train.next_batch(batch_size)
            if i % 100 == 0:
                train_accuracy = accuracy_calc.eval(feed_dict={x:batch[0], t:batch[1], keep_prob:1.0})
                print("Epoch: %d, Training Accuracy: %g" % (i, train_accuracy))
            training_algo.run(feed_dict={x:batch[0], t:batch[1], keep_prob:0.5})
            
        test_acc = 0
        #test_acc = accuracy_calc.eval(feed_dict={x:mnist.test.images, t:mnist.test.labels, keep_prob:1.0})
        #print("Test Accuracy: %g" % test_acc)

### Run the training...

In [ ]:
run_convolutional_mnist_model(training_algo, accuracy_calc, 100)

Epoch: 0, Training Accuracy: 0.13
Epoch: 100, Training Accuracy: 0.88
Epoch: 200, Training Accuracy: 0.92

## Git Update Shit

In [ ]:
%%bash
git add .
git pull
git commit -m "autocommit message"
git push -u origin master